## 4.4 업종별 리스크 분석한국표준산업분류(10차)를 활용한 업종별 부도 패턴 및 재무 리스크 분석

## 5. 신용정보 기반 부도 예측력 분석

In [246]:
# 산업분류 매핑 테이블 로드
mapping = pd.read_csv('../data/ksic_mapping.csv')

# 기업 데이터와 업종 매핑
df_industry = df.merge(mapping, left_on='업종(중분류)', right_on='업종코드', how='left')

print("=== 업종 매핑 결과 ===")
print(f"매핑 성공: {df_industry['대분류코드'].notna().sum():,}건 ({df_industry['대분류코드'].notna().sum()/len(df)*100:.1f}%)")
print(f"매핑 실패: {df_industry['대분류코드'].isna().sum():,}건")

print("\n=== 대분류별 기업 수 ===")
print(df_industry['대분류명'].value_counts())

=== 업종 매핑 결과 ===
매핑 성공: 50,100건 (100.0%)
매핑 실패: 5건

=== 대분류별 기업 수 ===
대분류명
제조업(10~34)                         18687
도매 및 소매업(45~47)                    10942
건설업(41~42)                          6277
전문, 과학 및 기술 서비스업(70~73)             2563
부동산업(68)                            2459
정보통신업(58~63)                        2344
사업시설 관리, 사업 지원 및 임대 서비스업(74~76)     1531
운수 및 창고업(49~52)                     1433
숙박 및 음식점업(55~56)                     837
협회 및 단체, 수리 및 기타 개인 서비스업(94~96)      681
예술, 스포츠 및 여가관련 서비스업(90~91)           562
수도, 하수 및 폐기물 처리, 원료 재생업(36~39)       512
농업, 임업 및 어업(01~03)                   454
교육 서비스업(85)                          383
전기, 가스, 증기 및 공기 조절 공급업(35)           172
보건업 및 사회복지 서비스업(86~87)               147
광업(05~08)                            104
금융 및 보험업(64~66)                        9
공공 행정, 국방 및 사회보장 행정(84)                3
Name: count, dtype: int64


### 4.4.1 업종별 부도율 분석

In [247]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 대분류별 부도율
major_default = df_industry.groupby('대분류명').agg({
    target_col: ['sum', 'count', 'mean']
}).round(4)
major_default.columns = ['부도기업수', '전체기업수', '부도율']
major_default = major_default.sort_values('부도율', ascending=False)

print("=== 대분류별 부도율 ===")
print(major_default)

# 중분류별 부도율 (상위 20개)
minor_default = df_industry.groupby(['대분류명', '중분류명']).agg({
    target_col: ['sum', 'count', 'mean']
}).round(4)
minor_default.columns = ['부도기업수', '전체기업수', '부도율']
minor_default = minor_default[minor_default['전체기업수'] >= 50]  # 최소 50개 기업 이상
minor_default = minor_default.sort_values('부도율', ascending=False)

print("\n=== 중분류별 부도율 (상위 20개, 최소 50개 기업) ===")
print(minor_default.head(20))

=== 대분류별 부도율 ===
                                 부도기업수  전체기업수   부도율
대분류명                                               
농업, 임업 및 어업(01~03)                  15    454 0.033
부동산업(68)                            71   2459 0.029
숙박 및 음식점업(55~56)                    21    837 0.025
예술, 스포츠 및 여가관련 서비스업(90~91)          12    562 0.021
건설업(41~42)                         106   6277 0.017
전문, 과학 및 기술 서비스업(70~73)             40   2563 0.016
도매 및 소매업(45~47)                    158  10942 0.014
운수 및 창고업(49~52)                     20   1433 0.014
제조업(10~34)                         256  18687 0.014
교육 서비스업(85)                          5    383 0.013
전기, 가스, 증기 및 공기 조절 공급업(35)           2    172 0.012
정보통신업(58~63)                        27   2344 0.011
사업시설 관리, 사업 지원 및 임대 서비스업(74~76)     15   1531 0.010
수도, 하수 및 폐기물 처리, 원료 재생업(36~39)       5    512 0.010
광업(05~08)                            1    104 0.010
보건업 및 사회복지 서비스업(86~87)               1    147 0.007
협회 및 단체, 수리 및 기타 개인 서비스업(94~96)      4    681 0

In [248]:
# 대분류별 부도율 시각화
fig = go.Figure()

fig.add_trace(go.Bar(
    x=major_default.index,
    y=major_default['부도율'] * 100,
    text=[f"{v:.2f}%" for v in major_default['부도율'] * 100],
    textposition='outside',
    marker_color='indianred',
    name='부도율'
))

fig.update_layout(
    title='대분류별 부도율',
    xaxis_title='산업 대분류',
    yaxis_title='부도율 (%)',
    height=500,
    font=dict(family='Malgun Gothic', size=12),
    xaxis={'tickangle': -45}
)

fig.show()

In [249]:
# 중분류별 부도율 시각화 (상위 20개)
top_20_minor = minor_default.head(20)

# 인덱스를 문자열로 변환 (대분류명 + 중분류명)
labels = [f"{major}\n{minor}" for major, minor in top_20_minor.index]

fig = go.Figure()

fig.add_trace(go.Bar(
    x=labels,
    y=top_20_minor['부도율'] * 100,
    text=[f"{v:.2f}%" for v in top_20_minor['부도율'] * 100],
    textposition='outside',
    marker_color='coral',
    name='부도율'
))

fig.update_layout(
    title='중분류별 부도율 Top 20 (최소 50개 기업 이상)',
    xaxis_title='산업 중분류',
    yaxis_title='부도율 (%)',
    height=600,
    font=dict(family='Malgun Gothic', size=10),
    xaxis={'tickangle': -45}
)

fig.show()

### 4.4.2 업종별 재무지표 리스크 분석

In [250]:
# 주요 재무지표별 업종 평균
key_ratios = [
    '재무비율_부채비율',
    '재무비율_유동비율',
    '재무비율_자기자본비율',
    '재무비율_영업이익율',
    '재무비율_당기순이익율',
    '재무비율_자기자본이익률(ROE)',
    '당좌비율',
    '이자보상배율'
]

# 대분류별 재무지표 평균 (부도기업 vs 정상기업)
industry_risk = {}

for ratio in key_ratios:
    if ratio in df_industry.columns:
        industry_comparison = df_industry.groupby(['대분류명', target_col])[ratio].agg(['mean', 'median', 'std']).round(2)
        industry_risk[ratio] = industry_comparison
        
        print(f"\n=== {ratio} - 대분류별 비교 ===")
        print(industry_comparison)


=== 재무비율_부채비율 - 대분류별 비교 ===
                                                                  mean  \
대분류명                            모형개발용Performance(향후1년내부도여부)              
건설업(41~42)                      0                             -524.600   
                                1                              371.210   
공공 행정, 국방 및 사회보장 행정(84)         0                              366.080   
광업(05~08)                       0                             1113.370   
                                1                              -52.520   
교육 서비스업(85)                     0                              209.770   
                                1                              -23.890   
금융 및 보험업(64~66)                 0                             1010.430   
농업, 임업 및 어업(01~03)              0                               60.330   
                                1                              187.560   
도매 및 소매업(45~47)                 0                             -197.790   
         

In [251]:
# 업종별 부채비율 비교 (부도 vs 정상)
if '재무비율_부채비율' in df_industry.columns:
    debt_ratio_by_industry = df_industry.groupby(['대분류명', target_col])['재무비율_부채비율'].median().unstack()
    
    fig = go.Figure()
    
    fig.add_trace(go.Bar(
        name='정상기업',
        x=debt_ratio_by_industry.index,
        y=debt_ratio_by_industry[0],
        marker_color='lightblue'
    ))
    
    fig.add_trace(go.Bar(
        name='부도기업',
        x=debt_ratio_by_industry.index,
        y=debt_ratio_by_industry[1],
        marker_color='darkred'
    ))
    
    fig.update_layout(
        title='업종별 부채비율 비교 (중앙값)',
        xaxis_title='산업 대분류',
        yaxis_title='부채비율 (%)',
        barmode='group',
        height=500,
        font=dict(family='Malgun Gothic', size=12),
        xaxis={'tickangle': -45}
    )
    
    fig.show()

In [252]:
# 업종별 유동비율 비교 (부도 vs 정상)
if '재무비율_유동비율' in df_industry.columns:
    current_ratio_by_industry = df_industry.groupby(['대분류명', target_col])['재무비율_유동비율'].median().unstack()
    
    fig = go.Figure()
    
    fig.add_trace(go.Bar(
        name='정상기업',
        x=current_ratio_by_industry.index,
        y=current_ratio_by_industry[0],
        marker_color='lightgreen'
    ))
    
    fig.add_trace(go.Bar(
        name='부도기업',
        x=current_ratio_by_industry.index,
        y=current_ratio_by_industry[1],
        marker_color='orange'
    ))
    
    fig.update_layout(
        title='업종별 유동비율 비교 (중앙값)',
        xaxis_title='산업 대분류',
        yaxis_title='유동비율 (%)',
        barmode='group',
        height=500,
        font=dict(family='Malgun Gothic', size=12),
        xaxis={'tickangle': -45}
    )
    
    fig.show()

### 4.4.3 업종별 유동성 위기 분석

In [253]:
# 유동성 지표
liquidity_cols = [
    '재무비율_유동비율',
    '당좌비율',
    '순운전자본',
    '현금',
    '현금등가물',
    '현금성자산'
]

available_liquidity = [col for col in liquidity_cols if col in df_industry.columns]

if available_liquidity:
    # 대분류별 유동성 지표 평균
    liquidity_by_industry = df_industry.groupby(['대분류명', target_col])[available_liquidity].median().round(2)
    
    print("=== 업종별 유동성 지표 (중앙값) ===")
    print(liquidity_by_industry)

=== 업종별 유동성 지표 (중앙값) ===
                                                             재무비율_유동비율  당좌비율  \
대분류명                            모형개발용Performance(향후1년내부도여부)                    
건설업(41~42)                      0                              292.830 2.300   
                                1                              229.230 1.670   
공공 행정, 국방 및 사회보장 행정(84)         0                               89.690 0.860   
광업(05~08)                       0                              191.230 1.730   
                                1                                3.640 0.040   
교육 서비스업(85)                     0                              145.340 1.170   
                                1                              113.000 0.370   
금융 및 보험업(64~66)                 0                              162.790 1.550   
농업, 임업 및 어업(01~03)              0                              154.720 1.180   
                                1                              160.100 1.220   
도매 및 소매업(45~47)

In [254]:
# 업종별 유동성 히트맵 (텍스트 제거 버전)
if len(available_liquidity) >= 2:
    # 정상기업 기준 업종별 유동성 지표
    normal_liquidity = df_industry[df_industry[target_col] == 0].groupby('대분류명')[available_liquidity].median()

    # 표준화 (0-1 스케일)
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    normal_liquidity_scaled = pd.DataFrame(
        scaler.fit_transform(normal_liquidity),
        columns=normal_liquidity.columns,
        index=normal_liquidity.index
    )

    fig = go.Figure(data=go.Heatmap(
        z=normal_liquidity_scaled.T.values,
        x=normal_liquidity_scaled.index,
        y=normal_liquidity_scaled.columns,
        colorscale='RdYlGn',
        text=normal_liquidity.T.values,
        hovertemplate='<b>%{y}</b><br>%{x}<br>중앙값: %{text:,.1f}<br>표준화: %{z:.3f}<extra></extra>'
    ))

    fig.update_layout(
        title='업종별 유동성 지표 히트맵 (정상기업 중앙값, 표준화)<br><sub>마우스를 올리면 상세 값을 확인할 수 있습니다</sub>',
        xaxis_title='산업 대분류',
        yaxis_title='유동성 지표',
        height=500,
        width=1200,
        font=dict(family='Malgun Gothic', size=11),
        xaxis={'tickangle': -45},
        margin=dict(l=150, r=50, t=100, b=150)  # 여백 조정
    )

    fig.show()

### 4.4.4 업종별 리스크 스코어

In [255]:
# 업종별 종합 리스크 스코어 계산
industry_risk_score = pd.DataFrame()

# 1. 부도율 (높을수록 위험)
industry_risk_score['부도율'] = major_default['부도율']

# 2. 평균 부채비율 (높을수록 위험)
if '재무비율_부채비율' in df_industry.columns:
    industry_risk_score['평균부채비율'] = df_industry.groupby('대분류명')['재무비율_부채비율'].median()

# 3. 평균 유동비율 (낮을수록 위험, 역수 취함)
if '재무비율_유동비율' in df_industry.columns:
    avg_current = df_industry.groupby('대분류명')['재무비율_유동비율'].median()
    industry_risk_score['유동비율역수'] = 1 / (avg_current / 100 + 0.01)  # 0으로 나누기 방지

# 4. 평균 영업이익률 (낮을수록 위험, 음수면 더 위험)
if '재무비율_영업이익율' in df_industry.columns:
    industry_risk_score['영업이익률'] = -df_industry.groupby('대분류명')['재무비율_영업이익율'].median()  # 음수 취함

# 표준화 및 종합 스코어
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
risk_scaled = pd.DataFrame(
    scaler.fit_transform(industry_risk_score.fillna(0)),
    columns=industry_risk_score.columns,
    index=industry_risk_score.index
)

# 종합 리스크 스코어 (평균)
industry_risk_score['종합리스크스코어'] = risk_scaled.mean(axis=1)
industry_risk_score = industry_risk_score.sort_values('종합리스크스코어', ascending=False)

print("=== 업종별 종합 리스크 스코어 ===")
print(industry_risk_score.round(3))

=== 업종별 종합 리스크 스코어 ===
                                  부도율  평균부채비율  유동비율역수  영업이익률  종합리스크스코어
대분류명                                                                  
공공 행정, 국방 및 사회보장 행정(84)         0.000 490.443   1.103  4.287     1.822
숙박 및 음식점업(55~56)                0.025 133.760   1.170 -0.000     1.086
예술, 스포츠 및 여가관련 서비스업(90~91)      0.021 103.060   1.417 -3.548     0.685
부동산업(68)                        0.029 143.176   0.608 -1.816     0.472
농업, 임업 및 어업(01~03)              0.033 142.181   0.641 -3.180     0.455
운수 및 창고업(49~52)                 0.014 126.505   0.746 -1.718     0.124
보건업 및 사회복지 서비스업(86~87)          0.007 139.685   0.922 -1.828     0.101
전기, 가스, 증기 및 공기 조절 공급업(35)      0.012 158.760   0.941 -5.188    -0.097
사업시설 관리, 사업 지원 및 임대 서비스업(74~76) 0.010 109.544   0.593 -1.393    -0.153
교육 서비스업(85)                     0.013  93.578   0.684 -3.003    -0.213
도매 및 소매업(45~47)                 0.014 131.815   0.533 -3.282    -0.242
제조업(10~34)                      0.014 124.962   0.606 

In [256]:
# 종합 리스크 스코어 시각화
fig = go.Figure()

colors = ['red' if score > 0 else 'green' for score in industry_risk_score['종합리스크스코어']]

fig.add_trace(go.Bar(
    x=industry_risk_score.index,
    y=industry_risk_score['종합리스크스코어'],
    marker_color=colors,
    text=[f"{v:.2f}" for v in industry_risk_score['종합리스크스코어']],
    textposition='outside'
))

fig.update_layout(
    title='업종별 종합 리스크 스코어 (표준화)',
    xaxis_title='산업 대분류',
    yaxis_title='종합 리스크 스코어',
    height=500,
    font=dict(family='Malgun Gothic', size=12),
    xaxis={'tickangle': -45}
)

fig.show()

### 4.4.5 업종별 인사이트 요약

In [257]:
print("\n" + "="*80)
print("📊 업종별 리스크 분석 - 핵심 인사이트")
print("="*80)

# 1. 부도율 최고/최저 업종
print("\n1️⃣ 부도율 분석")
print(f"   - 최고 부도율: {major_default.index[0]} ({major_default['부도율'].iloc[0]*100:.2f}%)")
print(f"   - 최저 부도율: {major_default.index[-1]} ({major_default['부도율'].iloc[-1]*100:.2f}%)")
print(f"   - 전체 평균: {major_default['부도율'].mean()*100:.2f}%")

# 2. 리스크 스코어 최고/최저 업종
print("\n2️⃣ 종합 리스크 스코어")
print(f"   - 최고 위험: {industry_risk_score.index[0]} (스코어: {industry_risk_score['종합리스크스코어'].iloc[0]:.3f})")
print(f"   - 최저 위험: {industry_risk_score.index[-1]} (스코어: {industry_risk_score['종합리스크스코어'].iloc[-1]:.3f})")

# 3. 업종별 기업 분포
print("\n3️⃣ 업종별 기업 분포 (상위 3개)")
top_3_industries = df_industry['대분류명'].value_counts().head(3)
for idx, (industry, count) in enumerate(top_3_industries.items(), 1):
    pct = count / len(df_industry) * 100
    print(f"   {idx}. {industry}: {count:,}개 기업 ({pct:.1f}%)")

# 4. 재무지표 특성
if '재무비율_부채비율' in df_industry.columns:
    print("\n4️⃣ 재무 특성")
    high_debt = df_industry.groupby('대분류명')['재무비율_부채비율'].median().idxmax()
    high_debt_val = df_industry.groupby('대분류명')['재무비율_부채비율'].median().max()
    print(f"   - 부채비율 최고 업종: {high_debt} ({high_debt_val:.1f}%)")
    
if '재무비율_유동비율' in df_industry.columns:
    low_liquidity = df_industry.groupby('대분류명')['재무비율_유동비율'].median().idxmin()
    low_liquidity_val = df_industry.groupby('대분류명')['재무비율_유동비율'].median().min()
    print(f"   - 유동비율 최저 업종: {low_liquidity} ({low_liquidity_val:.1f}%)")

print("\n" + "="*80)


📊 업종별 리스크 분석 - 핵심 인사이트

1️⃣ 부도율 분석
   - 최고 부도율: 농업, 임업 및 어업(01~03) (3.30%)
   - 최저 부도율: 금융 및 보험업(64~66) (0.00%)
   - 전체 평균: 1.37%

2️⃣ 종합 리스크 스코어
   - 최고 위험: 공공 행정, 국방 및 사회보장 행정(84) (스코어: 1.822)
   - 최저 위험: 금융 및 보험업(64~66) (스코어: -0.792)

3️⃣ 업종별 기업 분포 (상위 3개)
   1. 제조업(10~34): 18,687개 기업 (37.3%)
   2. 도매 및 소매업(45~47): 10,942개 기업 (21.8%)
   3. 건설업(41~42): 6,277개 기업 (12.5%)

4️⃣ 재무 특성
   - 부채비율 최고 업종: 공공 행정, 국방 및 사회보장 행정(84) (490.4%)
   - 유동비율 최저 업종: 예술, 스포츠 및 여가관련 서비스업(90~91) (69.6%)



In [258]:
# 신용정보 관련 컬럼 분석
credit_cols = [col for col in df.columns if any(keyword in col for keyword in 
                                                ['연체', '체납', '신용평가등급', '압류', '소송'])]

if credit_cols and target_col in df.columns:
    print("\n💳 신용정보 기반 부도 예측력 분석")
    print("="*60)
    
    # 연체 관련 변수 분석
    delinquency_cols = [col for col in credit_cols if '연체' in col]
    
    if delinquency_cols:
        # 연체 여부 생성
        df['연체여부'] = (df[delinquency_cols].sum(axis=1) > 0).astype(int)
        
        # 연체와 부도의 관계
        delinquency_bankruptcy = pd.crosstab(df['연체여부'], df[target_col],
                                           rownames=['연체'], colnames=['부도'],
                                           normalize='columns') * 100
        
        print("\n연체 여부와 부도의 관계 (%)")
        print(delinquency_bankruptcy)
    
    # 신용등급 분석
    rating_cols = [col for col in credit_cols if '등급' in col]
    
    if rating_cols:
        rating_col = rating_cols[0]
        
        # 신용등급별 부도율
        rating_bankruptcy = df.groupby(rating_col)[target_col].agg(['count', 'sum', 'mean'])
        rating_bankruptcy.columns = ['기업수', '부도기업수', '부도율']
        rating_bankruptcy['부도율(%)'] = rating_bankruptcy['부도율'] * 100
        rating_bankruptcy = rating_bankruptcy.sort_values('부도율(%)', ascending=False)
        
        print(f"\n신용등급별 부도율 (상위 10개)")
        print(rating_bankruptcy.head(10))
        
        # 시각화
        if len(rating_bankruptcy) > 0:
            fig = go.Figure()
            fig.add_trace(go.Bar(
                x=rating_bankruptcy.index[:15],
                y=rating_bankruptcy['부도율(%)'][:15],
                text=[f"{v:.2f}%" for v in rating_bankruptcy['부도율(%)'][:15]],
                textposition='outside',
                marker_color='crimson'
            ))
            
            fig.update_layout(
                title='신용등급별 부도율',
                xaxis_title='신용등급',
                yaxis_title='부도율 (%)',
                showlegend=False
            )
            fig.show()


💳 신용정보 기반 부도 예측력 분석

연체 여부와 부도의 관계 (%)
부도      0      1
연체              
0  95.380 57.576
1   4.620 42.424

신용등급별 부도율 (상위 10개)
                기업수  부도기업수   부도율  부도율(%)
기업신용평가등급(구간화)                           
10              715    107 0.150  14.965
9               774     91 0.118  11.757
8              2758    146 0.053   5.294
7              6053    177 0.029   2.924
6              7312    113 0.015   1.545
5              8296     70 0.008   0.844
3              6992     23 0.003   0.329
4              9211     26 0.003   0.282
1              3515      4 0.001   0.114
2              4443      2 0.000   0.045


## 6. 종합 리스크 스코어 생성

In [259]:
def create_risk_score(df):
    """도메인 지식 기반 종합 리스크 스코어 생성"""
    
    risk_score = pd.Series(index=df.index, dtype=float).fillna(0)
    
    # 1. 유동성 리스크 (30점)
    if '유동비율' in df.columns:
        risk_score += np.where(df['유동비율'] < 1, 10, 0)
        risk_score += np.where(df['유동비율'] < 0.5, 10, 0)
    
    if '현금비율' in df.columns:
        risk_score += np.where(df['현금비율'] < 0.1, 10, 0)
    
    # 2. 지급불능 리스크 (30점)
    if '부채비율' in df.columns:
        risk_score += np.where(df['부채비율'] > 200, 10, 0)
        risk_score += np.where(df['부채비율'] > 300, 10, 0)
    
    if '자본잠식여부' in df.columns:
        risk_score += np.where(df['자본잠식여부'] == 1, 10, 0)
    
    # 3. 수익성 리스크 (20점)
    if 'ROA' in df.columns:
        risk_score += np.where(df['ROA'] < 0, 10, 0)
    
    if '영업이익률' in df.columns:
        risk_score += np.where(df['영업이익률'] < 0, 10, 0)
    
    # 4. 신용 리스크 (20점)
    if '연체여부' in df.columns:
        risk_score += np.where(df['연체여부'] == 1, 20, 0)
    
    return risk_score

# 리스크 스코어 계산
df['종합리스크스코어'] = create_risk_score(df)

# 리스크 등급 분류
df['리스크등급'] = pd.cut(df['종합리스크스코어'], 
                       bins=[0, 20, 40, 60, 100],
                       labels=['안전', '주의', '경고', '위험'])

if target_col in df.columns:
    # 리스크 등급별 부도율
    risk_bankruptcy = df.groupby('리스크등급')[target_col].agg(['count', 'sum', 'mean'])
    risk_bankruptcy.columns = ['기업수', '부도기업수', '부도율']
    risk_bankruptcy['부도율(%)'] = risk_bankruptcy['부도율'] * 100
    
    print("\n🎯 종합 리스크 스코어 기반 부도율")
    print("="*60)
    print(risk_bankruptcy)
    
    # 시각화
    fig = go.Figure()
    
    colors = ['green', 'yellow', 'orange', 'red']
    fig.add_trace(go.Bar(
        x=risk_bankruptcy.index,
        y=risk_bankruptcy['부도율(%)'],
        text=[f"{v:.2f}%" for v in risk_bankruptcy['부도율(%)']],
        textposition='outside',
        marker_color=colors
    ))
    
    fig.update_layout(
        title='도메인 지식 기반 리스크 등급별 부도율',
        xaxis_title='리스크 등급',
        yaxis_title='부도율 (%)',
        showlegend=False
    )
    fig.show()
    
    # 리스크 스코어의 예측력 평가
    from sklearn.metrics import roc_auc_score, average_precision_score
    
    valid_idx = df['종합리스크스코어'].notna() & df[target_col].notna()
    if valid_idx.sum() > 0:
        auc = roc_auc_score(df.loc[valid_idx, target_col], 
                          df.loc[valid_idx, '종합리스크스코어'] / 100)
        pr_auc = average_precision_score(df.loc[valid_idx, target_col], 
                                       df.loc[valid_idx, '종합리스크스코어'] / 100)
        
        print(f"\n📊 도메인 기반 리스크 스코어 성능")
        print(f"ROC-AUC: {auc:.4f}")
        print(f"PR-AUC: {pr_auc:.4f}")


🎯 종합 리스크 스코어 기반 부도율
         기업수  부도기업수   부도율  부도율(%)
리스크등급                            
안전     21411    161 0.008   0.752
주의     15586    235 0.015   1.508
경고      6996    247 0.035   3.531
위험      1451    100 0.069   6.892



📊 도메인 기반 리스크 스코어 성능
ROC-AUC: 0.7077
PR-AUC: 0.0416


## 7. 핵심 인사이트 정리

In [260]:
print("\n" + "="*80)
print("📌 도메인 기반 부도 원인 분석 - 핵심 인사이트")
print("="*80)

insights = []

# 1. 불균형 비율
if target_col in df.columns:
    bankruptcy_rate = df[target_col].mean() * 100
    insights.append(f"• 전체 부도율: {bankruptcy_rate:.2f}% (심각한 불균형 데이터)")

# 2. 유동성 위기
if '유동비율' in df.columns and target_col in df.columns:
    low_liquidity_bankruptcy = df[df['유동비율'] < 1][target_col].mean() * 100
    insights.append(f"• 유동비율 1 미만 기업의 부도율: {low_liquidity_bankruptcy:.2f}%")

# 3. 자본잠식
if '자본잠식여부' in df.columns and target_col in df.columns:
    capital_erosion_bankruptcy = df[df['자본잠식여부'] == 1][target_col].mean() * 100
    insights.append(f"• 자본잠식 기업의 부도율: {capital_erosion_bankruptcy:.2f}%")

# 4. 수익성
if 'ROA' in df.columns and target_col in df.columns:
    negative_roa_bankruptcy = df[df['ROA'] < 0][target_col].mean() * 100
    insights.append(f"• ROA 마이너스 기업의 부도율: {negative_roa_bankruptcy:.2f}%")

# 5. 연체
if '연체여부' in df.columns and target_col in df.columns:
    delinquency_bankruptcy = df[df['연체여부'] == 1][target_col].mean() * 100
    insights.append(f"• 연체 이력 있는 기업의 부도율: {delinquency_bankruptcy:.2f}%")

for insight in insights:
    print(insight)

print("\n" + "="*80)
print("🎯 부도 예측 모델 개발 시 중점 사항")
print("="*80)

recommendations = [
    "1. 심각한 클래스 불균형 처리 필요 (SMOTE + Tomek Links 적용)",
    "2. 유동성 지표를 최우선 특성으로 활용",
    "3. 업종별 맞춤형 모델 고려",
    "4. 신용정보(연체, 체납) 변수의 높은 예측력 활용",
    "5. 도메인 지식 기반 복합 지표 생성 필요",
    "6. 재무조작 탐지 메커니즘 포함",
    "7. PR-AUC를 주요 평가 지표로 설정"
]

for rec in recommendations:
    print(rec)


📌 도메인 기반 부도 원인 분석 - 핵심 인사이트
• 전체 부도율: 1.51% (심각한 불균형 데이터)
• 유동비율 1 미만 기업의 부도율: 1.99%
• 자본잠식 기업의 부도율: 3.78%
• ROA 마이너스 기업의 부도율: 2.07%
• 연체 이력 있는 기업의 부도율: 12.38%

🎯 부도 예측 모델 개발 시 중점 사항
1. 심각한 클래스 불균형 처리 필요 (SMOTE + Tomek Links 적용)
2. 유동성 지표를 최우선 특성으로 활용
3. 업종별 맞춤형 모델 고려
4. 신용정보(연체, 체납) 변수의 높은 예측력 활용
5. 도메인 지식 기반 복합 지표 생성 필요
6. 재무조작 탐지 메커니즘 포함
7. PR-AUC를 주요 평가 지표로 설정


## 8. 다음 단계

이 분석을 바탕으로 다음 노트북에서는:
1. **고급 도메인 특성 공학**: 부도 예측에 특화된 복합 지표 생성
2. **상관관계 심층 분석**: 변수 간 상호작용 및 비선형 관계 포착
3. **불균형 분류 모델링**: SMOTE + Tomek Links 및 앙상블 기법 적용
4. **모델 평가 및 해석**: SHAP 분석 및 비즈니스 가치 최적화